In [1]:
import os
import json

import numpy as np
import pandas as pd

import statsmodels.api as sm
import statsmodels.formula.api as smf

from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
dataFol = os.getcwd() + "/dataFiles/"

In [4]:
pd.read_csv(dataFol + 'top10AprHospFeat.csv')

/nfshome/yg833/.conda/envs/mgPUI/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,Health Service Area,Facility Id,Length of Stay,APR DRG Code,APR DRG Description,APR Severity of Illness Code,Total Charges,Total Costs,yr,...,Hospital Ownership_Voluntary non-profit - Other,Hospital Ownership_Voluntary non-profit - Private,rural_versus_urban_R,rural_versus_urban_U,ratioInsuredTotalPopulation,ratioFertilityTotalPopulation,ratioChargesCosts,days,chargesPerDay,avgNumberDays
0,0,Western NY,37.0,8,139,Other pneumonia,3.0,5621.25,5037.17,2013,...,0,1,1,0,0.989499,0.218558,1.115954,8.0,702.656250,3.650000
1,1,Western NY,37.0,3,139,Other pneumonia,2.0,2435.75,2271.92,2013,...,0,1,1,0,0.989499,0.218558,1.072111,3.0,811.916667,3.650000
2,2,Western NY,37.0,4,139,Other pneumonia,2.0,3754.95,3579.69,2013,...,0,1,1,0,0.989499,0.218558,1.048960,4.0,938.737500,3.650000
3,3,Western NY,37.0,2,139,Other pneumonia,1.0,2096.05,2195.02,2013,...,0,1,1,0,0.989499,0.218558,0.954912,2.0,1048.025000,3.650000
4,4,Western NY,37.0,1,139,Other pneumonia,3.0,1583.25,1772.56,2013,...,0,1,1,0,0.989499,0.218558,0.893200,1.0,1583.250000,3.650000
5,5,Western NY,37.0,2,139,Other pneumonia,1.0,2776.20,2683.35,2013,...,0,1,1,0,0.989499,0.218558,1.034602,2.0,1388.100000,3.650000
6,6,Western NY,37.0,2,139,Other pneumonia,1.0,1740.60,1939.36,2013,...,0,1,1,0,0.989499,0.218558,0.897513,2.0,870.300000,3.650000
7,7,Western NY,37.0,1,139,Other pneumonia,2.0,552.85,510.55,2013,...,0,1,1,0,0.989499,0.218558,1.082852,1.0,552.850000,3.650000
8,8,Western NY,37.0,2,139,Other pneumonia,1.0,2337.65,2510.50,2013,...,0,1,1,0,0.989499,0.218558,0.931149,2.0,1168.825000,3.650000
9,9,Western NY,37.0,3,139,Other pneumonia,1.0,2695.15,2744.45,2013,...,0,1,1,0,0.989499,0.218558,0.982036,3.0,898.383333,3.650000


In [4]:
### check that the file is not already there
if not os.path.isfile(dataFol + "cms/ippsProviderSummary.csv"):
    if os.path.isfile("ippsProviderSummary.csv"):
        # if in the current dir just move it
        os.system("mv " + "ippsProviderSummary.csv " + dataFol+"cms/")
        print ("Moving file!, Please check!")
        #otherwise start looking for the zip file
    else:
        if not os.path.isfile("ippsProviderSummary.csv"):
            urllib.request.urlretrieve(ippsUrl, "ippsProviderSummary.csv")
            #os.system("curl -O ")
            ###  To move it I use the os.system() functions to run bash commands with arguments
            os.system("mv " + "ippsProviderSummary.csv " + dataFol+"cms/")
            ### One final check:

if not os.path.isfile(dataFol+ "cms/ippsProviderSummary.csv"):
    print ("WARNING!!! something is wrong: the file is not there!")
else:
    print ("file in place, you can continue")

file in place, you can continue


In [6]:
ippsRaw = pd.read_csv(dataFol+"cms/ippsProviderSummary.csv")

In [11]:
ippsRaw.head()

,DRG Definition,Provider Id,Provider Name,Provider Street Address,Provider City,Provider State,Provider Zip Code,Hospital Referral Region (HRR) Description,Total Discharges,Average Covered Charges,Average Total Payments,Average Medicare Payments,drg,drgDesc
0,001 - HEART TRANSPLANT OR IMPLANT OF HEART ASS...,10033,UNIVERSITY OF ALABAMA HOSPITAL,619 SOUTH 19TH STREET,BIRMINGHAM,AL,35233,AL - Birmingham,13,1016806.46,296937.00,150139.69,001,HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SY...
1,001 - HEART TRANSPLANT OR IMPLANT OF HEART ASS...,30103,MAYO CLINIC HOSPITAL,5777 EAST MAYO BOULEVARD,PHOENIX,AZ,85054,AZ - Phoenix,26,443387.54,215059.54,163889.31,001,HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SY...
2,001 - HEART TRANSPLANT OR IMPLANT OF HEART ASS...,40114,BAPTIST HEALTH MEDICAL CENTER-LITTLE ROCK,"9601 INTERSTATE 630, EXIT 7",LITTLE ROCK,AR,72205,AR - Little Rock,33,711472.00,180315.55,145192.61,001,HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SY...
3,001 - HEART TRANSPLANT OR IMPLANT OF HEART ASS...,50025,UC SAN DIEGO HEALTH HILLCREST - HILLCREST MED CTR,200 WEST ARBOR DRIVE,SAN DIEGO,CA,92103,CA - San Diego,17,796343.82,299244.41,270131.59,001,HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SY...
4,001 - HEART TRANSPLANT OR IMPLANT OF HEART ASS...,50100,SHARP MEMORIAL HOSPITAL,7901 FROST ST,SAN DIEGO,CA,92123,CA - San Diego,13,1434651.46,239537.46,215205.00,001,HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SY...


In [9]:
ippsRaw.columns

Index(['DRG Definition', 'Provider Id', 'Provider Name',
       'Provider Street Address', 'Provider City', 'Provider State',
       'Provider Zip Code', 'Hospital Referral Region (HRR) Description',
       'Total Discharges', 'Average Covered Charges', 'Average Total Payments',
       'Average Medicare Payments'],
      dtype='object')

In [12]:
ippsRaw.dtypes

DRG Definition                                 object
Provider Id                                     int64
Provider Name                                  object
Provider Street Address                        object
Provider City                                  object
Provider State                                 object
Provider Zip Code                               int64
Hospital Referral Region (HRR) Description     object
Total Discharges                                int64
Average Covered Charges                       float64
Average Total Payments                        float64
Average Medicare Payments                     float64
drg                                            object
drgDesc                                        object
dtype: object

In [10]:
# split the DRG definition column into code and description
ippsRaw['drg'], ippsRaw['drgDesc'] = ippsRaw['DRG Definition'].str.split(' - ', 1).str

In [17]:
ipps = ippsRaw[ippsRaw['Provider State'] == "NY"][['Provider Id', 'drg', 'drgDesc',
         'Total Discharges', 'Average Covered Charges', 
         'Average Total Payments', 'Average Medicare Payments']]

In [20]:
ipps.head()

,Provider Id,drg,drgDesc,Total Discharges,Average Covered Charges,Average Total Payments,Average Medicare Payments
44,330024,001,HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SY...,23,505030.22,279821.43,236846.35
45,330059,001,HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SY...,16,771758.13,283074.31,269391.38
46,330101,001,HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SY...,44,1091512.32,334808.75,297501.18
47,330234,001,HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SY...,14,1781830.21,358592.36,328005.64
48,330285,001,HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SY...,23,560964.13,267191.74,196727.52


In [26]:
nycHosp = pd.read_csv(dataFol+"compiledMasterchargesNycHospitals.csv")

In [27]:
nycHosp.head()

,Unnamed: 0,drgDesc,avgCharges,providerId,drg,msDrg,numDischarges,Charge No,Charge Description,Standard Charges,...,Cases,SITE,DEPT,SITE CHRG,SITE D DESCRIPTION,HCPCS CD,SITE RATE,REV,OMCR,COUNT
0,0,ISOLATION ROOM,"1,244.00",330204.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,MS 1ST HR OBS ISOLATION,713.00,330204.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,MS 1ST HR OBS TELEMETRY,809.00,330204.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,MS ADD HR OBS ISOLATION,21.00,330204.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,MS ADD OBS TELEMETRY,24.00,330204.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
